<a href="https://colab.research.google.com/github/huune2k/BTVN_AI_8-8-2022/blob/main/VietNamMoney_CNN_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
from pathlib import Path

direction = Path('/content/gdrive/MyDrive/AI_TranTrongHuu_18146132/VietNamMoney_Data')

In [ ]:
# create dataset from file list 
filelist_ds = tf.data.Dataset.list_files(str(direction/'*/*'))

ds_size = 540 #number of img
train_size = int(ds_size*0.8)
val_size = int(ds_size*0.2)

filelist_ds = filelist_ds.shuffle(ds_size) #trộn
train_file = filelist_ds.take(train_size) 
val_file = filelist_ds.take(val_size) 

for file in filelist_ds.take(3):
  print(file)

tf.Tensor(b'/content/gdrive/MyDrive/AI_TranTrongHuu_18146132/VietNamMoney_Data/2/2K 1.jpeg', shape=(), dtype=string)
tf.Tensor(b'/content/gdrive/MyDrive/AI_TranTrongHuu_18146132/VietNamMoney_Data/50/50K 57.jpeg', shape=(), dtype=string)
tf.Tensor(b'/content/gdrive/MyDrive/AI_TranTrongHuu_18146132/VietNamMoney_Data/10/10K 56.jpeg', shape=(), dtype=string)


In [ ]:
# gán label from dataset
def get_label(file_path):
  labels = []
  parts = tf.strings.split(file_path,'/')
  text = parts[-2]
  text_split = tf.strings.split(text,'.')
  number_label = int(text_split[0])
  labels = [1 if x == number_label else 0 for x in range(1,10,1)]
  return tf.convert_to_tensor(labels) 

In [ ]:
#preprocessing
img_h = 100
img_w = 200

def preprocessing(file_path):
  #read file
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)

  #convert from unit8 to float32 and normalize value to[0,1]
  img = tf.image.convert_image_dtype(img, tf.float32)

  #resize
  img = tf.image.resize(img,[img_h, img_w])

  #get image label
  label = get_label(file_path)
  return img, label

In [ ]:
import numpy as np

batch_size = 32

# Build train dataset
train_ds = filelist_ds.shuffle(train_size)
train_ds = train_ds.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
#train_ds = train_ds.map(lambda x:tf.function(preprocessing, [x], [tf.float32,tf.int32]))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(1)

# Build val dataset
val_ds = filelist_ds.shuffle(val_size)
val_ds = val_ds.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
#val_ds = val_ds.map(lambda x:tf.function(preprocessing, [x], [tf.float32,tf.int32]))
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(1)
''' 
for file in train_ds.take(3):
  print(file)  '''

print(train_ds)
print(val_ds)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 9), dtype=tf.int32, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 9), dtype=tf.int32, name=None))>


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(224,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(100,200,3)))
model.add(Conv2D(224,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(112,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(112,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(32,activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(9,activation='softmax'))

model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_155 (Conv2D)         (None, 100, 200, 224)     6272      
                                                                 
 conv2d_156 (Conv2D)         (None, 100, 200, 224)     451808    
                                                                 
 max_pooling2d_99 (MaxPoolin  (None, 50, 100, 224)     0         
 g2D)                                                            
                                                                 
 conv2d_157 (Conv2D)         (None, 50, 100, 112)      225904    
                                                                 
 conv2d_158 (Conv2D)         (None, 50, 100, 112)      113008    
                                                                 
 max_pooling2d_100 (MaxPooli  (None, 25, 50, 112)      0         
 ng2D)                                               

In [ ]:
# compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

In [ ]:
# train model
model.fit(train_ds, validation_data=val_ds, epochs=5)

In [ ]:
import pickle
file_name = "moneymodel.sav"
pickle.dump(model, open(file_name, 'wb'))

INFO:tensorflow:Assets written to: ram://8184f8f4-1b17-42d2-a0ce-e79a814ad8c8/assets
